# FINMA Python Lab 08 - Solutions

This notebook contains complete solutions for all exercises in Lab 8.

In [ ]:
# Import libraries
import numpy as np
import pandas as pd

---

## Exercise 1: Portfolio Value Calculation (NumPy)

In [ ]:
# Exercise 1: Portfolio Value Calculation

# Data
symbols = np.array(['AAPL', 'GOOGL', 'MSFT', 'AMZN', 'TSLA'])
shares = np.array([150, 75, 100, 60, 40])
prices = np.array([163.75, 155.30, 406.20, 200.45, 276.30])

# 1. Calculate value of each position
position_values = shares * prices
print("Position Values:")
for symbol, value in zip(symbols, position_values):
    print(f"{symbol}: ${value:,.2f}")

# 2. Calculate total portfolio value
total_value = np.sum(position_values)
print(f"\nTotal Portfolio Value: ${total_value:,.2f}")

# 3. Calculate percentage allocation
percentages = (position_values / total_value) * 100
print("\nPercentage Allocation:")
for symbol, pct in zip(symbols, percentages):
    print(f"{symbol}: {pct:.2f}%")

---

## Exercise 2: Return Statistics (NumPy)

In [ ]:
# Exercise 2: Return Statistics

# Daily returns (in percentage)
returns = np.array([1.2, -0.5, 0.8, 1.5, -0.3, 0.9, 1.1, -0.7, 1.3, 0.6])

# 1. Average daily return
avg_daily_return = np.mean(returns)
print(f"Average Daily Return: {avg_daily_return:.4f}%")

# 2. Volatility (standard deviation)
daily_volatility = np.std(returns, ddof=1)  # ddof=1 for sample std
print(f"Daily Volatility: {daily_volatility:.4f}%")

# 3. Annualized return (252 trading days)
annualized_return = avg_daily_return * 252
print(f"Annualized Return: {annualized_return:.2f}%")

# 4. Annualized volatility
annualized_volatility = daily_volatility * np.sqrt(252)
print(f"Annualized Volatility: {annualized_volatility:.2f}%")

# 5. Sharpe ratio (risk-free rate = 2%)
risk_free_rate = 2.0
sharpe_ratio = (annualized_return - risk_free_rate) / annualized_volatility
print(f"Sharpe Ratio: {sharpe_ratio:.4f}")

---

## Exercise 3: Price Matrix Analysis (NumPy)

In [ ]:
# Exercise 3: Price Matrix Analysis

# Read first 10 days of data
df = pd.read_csv('sample_data_lab8/stock_prices_timeseries.csv')
price_matrix = df.iloc[:10, 1:].values  # First 10 days, all stocks except Date

symbols = df.columns[1:].tolist()

print("Price Matrix Shape:", price_matrix.shape)
print("\nPrice Matrix:")
print(price_matrix)

# 1. Average price for each stock
avg_prices = np.mean(price_matrix, axis=0)
print("\n1. Average Price by Stock:")
for symbol, avg in zip(symbols, avg_prices):
    print(f"{symbol}: ${avg:.2f}")

# 2. Highest price for each stock
max_prices = np.max(price_matrix, axis=0)
print("\n2. Highest Price by Stock:")
for symbol, max_p in zip(symbols, max_prices):
    print(f"{symbol}: ${max_p:.2f}")

# 3. Lowest price for each stock
min_prices = np.min(price_matrix, axis=0)
print("\n3. Lowest Price by Stock:")
for symbol, min_p in zip(symbols, min_prices):
    print(f"{symbol}: ${min_p:.2f}")

# 4. Stock with highest average price
highest_avg_idx = np.argmax(avg_prices)
print(f"\n4. Stock with Highest Average: {symbols[highest_avg_idx]} (${avg_prices[highest_avg_idx]:.2f})")

# 5. Day with highest average price
avg_by_day = np.mean(price_matrix, axis=1)
highest_day_idx = np.argmax(avg_by_day)
print(f"\n5. Day with Highest Average: Day {highest_day_idx + 1} (${avg_by_day[highest_day_idx]:.2f})")

---

## Exercise 4: DataFrame Creation and Analysis (Pandas)

In [ ]:
# Exercise 4: DataFrame Creation and Analysis

# Read company info
companies = pd.read_csv('sample_data_lab8/company_info.csv')

# 1. Display first 5 rows
print("1. First 5 rows:")
print(companies.head())

# 2. Show summary statistics
print("\n2. Summary Statistics:")
print(companies.describe())

# 3. Company with highest market cap
highest_mc = companies.loc[companies['market_cap'].idxmax()]
print(f"\n3. Highest Market Cap: {highest_mc['company']} (${highest_mc['market_cap']:.1f}B)")

# 4. Company with highest P/E ratio
highest_pe = companies.loc[companies['pe_ratio'].idxmax()]
print(f"\n4. Highest P/E Ratio: {highest_pe['company']} ({highest_pe['pe_ratio']:.1f})")

# 5. Average dividend yield by sector
print("\n5. Average Dividend Yield by Sector:")
avg_dividend = companies.groupby('sector')['dividend_yield'].mean()
print(avg_dividend)

# 6. List all Technology sector companies
print("\n6. Technology Sector Companies:")
tech_companies = companies[companies['sector'] == 'Technology']
print(tech_companies[['symbol', 'company', 'industry']])

---

## Exercise 5: Transaction Analysis (Pandas)

In [ ]:
# Exercise 5: Transaction Analysis

# Read transactions
transactions = pd.read_csv('sample_data_lab8/transactions.csv')

# 1. Calculate total shares bought and sold
print("1. Shares Bought and Sold by Symbol:")
summary = transactions.groupby(['symbol', 'action'])['shares'].sum().unstack(fill_value=0)
print(summary)

# 2. Calculate total commission paid
total_commission = transactions['commission'].sum()
print(f"\n2. Total Commission Paid: ${total_commission:.2f}")

# 3. Calculate average buy and sell prices
print("\n3. Average Prices:")
avg_prices = transactions.groupby(['symbol', 'action']).agg({
    'price': 'mean',
    'shares': 'sum'
})
print(avg_prices)

# 4. Determine current holdings
print("\n4. Current Holdings:")
buys = transactions[transactions['action'] == 'BUY'].groupby('symbol')['shares'].sum()
sells = transactions[transactions['action'] == 'SELL'].groupby('symbol')['shares'].sum()
holdings = buys.subtract(sells, fill_value=0)
print(holdings)

# 5. Export summary to CSV
# Calculate metrics for export
avg_buy_prices = transactions[transactions['action'] == 'BUY'].groupby('symbol')['price'].mean()
commission_by_symbol = transactions.groupby('symbol')['commission'].sum()

export_df = pd.DataFrame({
    'symbol': holdings.index,
    'shares_held': holdings.values,
    'avg_buy_price': avg_buy_prices[holdings.index].values,
    'total_commission': commission_by_symbol[holdings.index].values
})

export_df.to_csv('output/holdings_summary.csv', index=False)
print("\n5. Summary exported to output/holdings_summary.csv")
print(export_df)

---

## Exercise 6: Time Series Returns (Pandas)

In [ ]:
# Exercise 6: Time Series Returns

# 1. Read data with Date as index
prices_df = pd.read_csv('sample_data_lab8/stock_prices_timeseries.csv')
prices_df['Date'] = pd.to_datetime(prices_df['Date'])
prices_df.set_index('Date', inplace=True)

print("Price Data:")
print(prices_df.head())

# 2. Calculate daily returns
returns = prices_df.pct_change() * 100  # Convert to percentage
print("\nDaily Returns (%):")
print(returns.head())

# 3. Calculate cumulative returns
cumulative_returns = (1 + prices_df.pct_change()).cumprod() - 1
total_returns = cumulative_returns.iloc[-1] * 100

print("\nTotal Returns (%):")
print(total_returns)

# 4. Find stocks with extreme characteristics
print("\n4. Stock Characteristics:")
print(f"Highest Total Return: {total_returns.idxmax()} ({total_returns.max():.2f}%)")
print(f"Lowest Total Return: {total_returns.idxmin()} ({total_returns.min():.2f}%)")

volatilities = returns.std()
print(f"Highest Volatility: {volatilities.idxmax()} ({volatilities.max():.2f}%)")
print(f"Lowest Volatility: {volatilities.idxmin()} ({volatilities.min():.2f}%)")

# 5. Export daily returns
returns.to_csv('output/daily_returns.csv')
print("\n5. Daily returns exported to output/daily_returns.csv")

---

## Exercise 7: Moving Averages (Pandas)

In [ ]:
# Exercise 7: Moving Averages

# Read data
prices_df = pd.read_csv('sample_data_lab8/stock_prices_timeseries.csv')
prices_df['Date'] = pd.to_datetime(prices_df['Date'])
prices_df.set_index('Date', inplace=True)

# 1. Calculate 5-day and 10-day moving averages for AAPL
prices_df['AAPL_MA5'] = prices_df['AAPL'].rolling(window=5).mean()
prices_df['AAPL_MA10'] = prices_df['AAPL'].rolling(window=10).mean()

print("AAPL with Moving Averages:")
print(prices_df[['AAPL', 'AAPL_MA5', 'AAPL_MA10']].head(15))

# 2 & 3. Identify crossover signals
# Buy signal: price crosses above MA5
# Sell signal: price crosses below MA5

prices_df['Signal'] = ''
for i in range(1, len(prices_df)):
    if pd.notna(prices_df['AAPL_MA5'].iloc[i]):
        # Check if price crosses above MA (buy signal)
        if (prices_df['AAPL'].iloc[i] > prices_df['AAPL_MA5'].iloc[i] and 
            prices_df['AAPL'].iloc[i-1] <= prices_df['AAPL_MA5'].iloc[i-1]):
            prices_df.loc[prices_df.index[i], 'Signal'] = 'BUY'
        # Check if price crosses below MA (sell signal)
        elif (prices_df['AAPL'].iloc[i] < prices_df['AAPL_MA5'].iloc[i] and 
              prices_df['AAPL'].iloc[i-1] >= prices_df['AAPL_MA5'].iloc[i-1]):
            prices_df.loc[prices_df.index[i], 'Signal'] = 'SELL'

# 4. Count signals
buy_signals = (prices_df['Signal'] == 'BUY').sum()
sell_signals = (prices_df['Signal'] == 'SELL').sum()

print(f"\nBuy Signals: {buy_signals}")
print(f"Sell Signals: {sell_signals}")

# Show days with signals
signals_df = prices_df[prices_df['Signal'] != '']
print("\nDays with Signals:")
print(signals_df[['AAPL', 'AAPL_MA5', 'Signal']])

# 5. Create final DataFrame
result_df = prices_df[['AAPL', 'AAPL_MA5', 'AAPL_MA10', 'Signal']].copy()
result_df.columns = ['Price', 'MA5', 'MA10', 'Signal']
result_df.to_csv('output/aapl_moving_averages.csv')
print("\nData exported to output/aapl_moving_averages.csv")

---

## Exercise 8: Portfolio Performance Report (Combined)

In [ ]:
# Exercise 8: Portfolio Performance Report

# 1. Read transactions to determine holdings
transactions = pd.read_csv('sample_data_lab8/transactions.csv')

# Calculate current holdings and cost basis
holdings_data = []
for symbol in transactions['symbol'].unique():
    symbol_trans = transactions[transactions['symbol'] == symbol]
    
    buys = symbol_trans[symbol_trans['action'] == 'BUY']
    sells = symbol_trans[symbol_trans['action'] == 'SELL']
    
    shares_bought = buys['shares'].sum()
    shares_sold = sells['shares'].sum()
    shares_held = shares_bought - shares_sold
    
    # Calculate weighted average buy price
    total_cost = (buys['shares'] * buys['price']).sum() + buys['commission'].sum()
    avg_buy_price = total_cost / shares_bought if shares_bought > 0 else 0
    
    holdings_data.append({
        'symbol': symbol,
        'shares': shares_held,
        'avg_buy_price': avg_buy_price,
        'cost_basis': shares_held * avg_buy_price
    })

holdings_df = pd.DataFrame(holdings_data)
holdings_df = holdings_df[holdings_df['shares'] > 0]  # Only keep positive holdings

# 2. Get latest prices
prices_df = pd.read_csv('sample_data_lab8/stock_prices_timeseries.csv')
latest_prices = prices_df.iloc[-1][['AAPL', 'GOOGL', 'MSFT', 'AMZN', 'TSLA']]
latest_prices_dict = latest_prices.to_dict()

# 3. Merge with company info
companies = pd.read_csv('sample_data_lab8/company_info.csv')
holdings_df = pd.merge(holdings_df, companies[['symbol', 'company', 'sector']], on='symbol')

# 4. Calculate metrics
holdings_df['current_price'] = holdings_df['symbol'].map(latest_prices_dict)
holdings_df['current_value'] = holdings_df['shares'] * holdings_df['current_price']
holdings_df['gain_loss'] = holdings_df['current_value'] - holdings_df['cost_basis']
holdings_df['return_pct'] = (holdings_df['gain_loss'] / holdings_df['cost_basis']) * 100

# 5. Portfolio-level metrics
total_value = holdings_df['current_value'].sum()
total_cost = holdings_df['cost_basis'].sum()
total_gain_loss = holdings_df['gain_loss'].sum()
overall_return = (total_gain_loss / total_cost) * 100

# Calculate allocation by sector
sector_allocation = holdings_df.groupby('sector')['current_value'].sum() / total_value * 100

# Display report
print("PORTFOLIO PERFORMANCE REPORT")
print("=" * 100)
print("\nHoldings:")
print(holdings_df.to_string(index=False))

print("\n" + "=" * 100)
print("PORTFOLIO SUMMARY")
print("=" * 100)
print(f"Total Current Value: ${total_value:,.2f}")
print(f"Total Cost Basis: ${total_cost:,.2f}")
print(f"Total Gain/Loss: ${total_gain_loss:,.2f}")
print(f"Overall Return: {overall_return:+.2f}%")

print("\nSector Allocation:")
for sector, pct in sector_allocation.items():
    print(f"  {sector}: {pct:.2f}%")

# 6. Export to CSV
holdings_df.to_csv('output/portfolio_performance.csv', index=False)
print("\nFull report exported to output/portfolio_performance.csv")

---

## Exercise 9: Correlation Analysis (Pandas + NumPy)

In [ ]:
# Exercise 9: Correlation Analysis

# 1. Read data
prices_df = pd.read_csv('sample_data_lab8/stock_prices_timeseries.csv')
prices_df['Date'] = pd.to_datetime(prices_df['Date'])
prices_df.set_index('Date', inplace=True)

# 2. Calculate daily returns
returns = prices_df.pct_change().dropna()

print("Daily Returns:")
print(returns.head())

# 3. Create correlation matrix
correlation_matrix = returns.corr()

print("\nCorrelation Matrix:")
print(correlation_matrix)

# 4. Find pairs with highest and lowest correlation
# Create mask to exclude diagonal and duplicates
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
corr_values = correlation_matrix.mask(mask)

# Find highest correlation (excluding 1.0 on diagonal)
max_corr = corr_values.max().max()
max_pair = corr_values.stack().idxmax()
print(f"\nHighest Correlation: {max_pair[0]} - {max_pair[1]}: {max_corr:.4f}")

# Find lowest correlation
min_corr = corr_values.min().min()
min_pair = corr_values.stack().idxmin()
print(f"Lowest Correlation: {min_pair[0]} - {min_pair[1]}: {min_corr:.4f}")

# 5. Calculate portfolio variance for equal-weighted portfolio
n_stocks = len(returns.columns)
weights = np.array([1/n_stocks] * n_stocks)  # Equal weights

# Covariance matrix
cov_matrix = returns.cov()

# Portfolio variance = w^T * Cov * w
portfolio_variance = np.dot(weights, np.dot(cov_matrix, weights))
portfolio_std = np.sqrt(portfolio_variance)

# Annualize (252 trading days)
annualized_volatility = portfolio_std * np.sqrt(252) * 100

print(f"\nEqual-Weighted Portfolio:")
print(f"Daily Volatility: {portfolio_std*100:.4f}%")
print(f"Annualized Volatility: {annualized_volatility:.2f}%")

# Export correlation matrix
correlation_matrix.to_csv('output/correlation_matrix.csv')
print("\nCorrelation matrix exported to output/correlation_matrix.csv")

---

## Exercise 10: Risk-Adjusted Returns (Advanced)

In [ ]:
# Exercise 10: Risk-Adjusted Returns

# Read data
prices_df = pd.read_csv('sample_data_lab8/stock_prices_timeseries.csv')
prices_df['Date'] = pd.to_datetime(prices_df['Date'])
prices_df.set_index('Date', inplace=True)

# 1. Calculate daily and annualized returns
returns = prices_df.pct_change().dropna()
mean_daily_returns = returns.mean()
annualized_returns = mean_daily_returns * 252 * 100  # Convert to percentage

# 2. Calculate volatility
daily_volatility = returns.std()
annualized_volatility = daily_volatility * np.sqrt(252) * 100

# 3. Calculate Sharpe Ratio
risk_free_rate = 2.0  # 2% annual
sharpe_ratios = (annualized_returns - risk_free_rate) / annualized_volatility

# 4. Calculate Maximum Drawdown
def calculate_max_drawdown(prices):
    """Calculate maximum drawdown for a price series"""
    cumulative_max = prices.cummax()
    drawdown = (prices - cumulative_max) / cumulative_max
    max_drawdown = drawdown.min()
    return max_drawdown * 100  # Convert to percentage

max_drawdowns = prices_df.apply(calculate_max_drawdown)

# 5. Calculate total returns
total_returns = ((prices_df.iloc[-1] / prices_df.iloc[0]) - 1) * 100

# Create summary DataFrame
summary = pd.DataFrame({
    'Symbol': prices_df.columns,
    'Total_Return_%': total_returns.values,
    'Annualized_Return_%': annualized_returns.values,
    'Annualized_Volatility_%': annualized_volatility.values,
    'Sharpe_Ratio': sharpe_ratios.values,
    'Max_Drawdown_%': max_drawdowns.values
})

# Rankings
print("RISK-ADJUSTED RETURNS ANALYSIS")
print("=" * 100)
print("\nComplete Metrics:")
print(summary.to_string(index=False))

print("\n" + "=" * 100)
print("RANKINGS")
print("=" * 100)

# Rank by total return
print("\nBy Total Return:")
print(summary.sort_values('Total_Return_%', ascending=False)[['Symbol', 'Total_Return_%']].to_string(index=False))

# Rank by Sharpe ratio
print("\nBy Sharpe Ratio (Risk-Adjusted):")
print(summary.sort_values('Sharpe_Ratio', ascending=False)[['Symbol', 'Sharpe_Ratio']].to_string(index=False))

# Rank by volatility (risk)
print("\nBy Risk (Volatility - Lower is Better):")
print(summary.sort_values('Annualized_Volatility_%')[['Symbol', 'Annualized_Volatility_%']].to_string(index=False))

# 6. Export analysis
summary.to_csv('output/risk_adjusted_returns.csv', index=False)
print("\nAnalysis exported to output/risk_adjusted_returns.csv")

# Additional insights
print("\n" + "=" * 100)
print("KEY INSIGHTS")
print("=" * 100)
best_return = summary.loc[summary['Total_Return_%'].idxmax()]
best_sharpe = summary.loc[summary['Sharpe_Ratio'].idxmax()]
lowest_risk = summary.loc[summary['Annualized_Volatility_%'].idxmin()]

print(f"\nBest Performer (Return): {best_return['Symbol']} ({best_return['Total_Return_%']:.2f}%)")
print(f"Best Risk-Adjusted (Sharpe): {best_sharpe['Symbol']} ({best_sharpe['Sharpe_Ratio']:.4f})")
print(f"Lowest Risk: {lowest_risk['Symbol']} ({lowest_risk['Annualized_Volatility_%']:.2f}%)")

---

## Summary

All exercises completed! Key takeaways:

1. **NumPy** excels at fast numerical operations on arrays
2. **Pandas** makes data manipulation and analysis intuitive
3. **Time series** analysis is crucial for financial data
4. **Risk metrics** (volatility, Sharpe ratio, drawdown) are essential
5. **Correlation** helps understand portfolio diversification
6. **Proper data structures** lead to cleaner code

### Files Created:
- `output/holdings_summary.csv`
- `output/daily_returns.csv`
- `output/aapl_moving_averages.csv`
- `output/portfolio_performance.csv`
- `output/correlation_matrix.csv`
- `output/risk_adjusted_returns.csv`